### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [18]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

## Player Count

* Display the total number of players


In [19]:
#get list of unique players
uniqueplayer = purchase_data["SN"].unique()
#get count of unqiue players (/length of list)
uniqueplayer_count = len(uniqueplayer)
#put count into dataframe to print
uniqueplayer_df = pd.DataFrame({"Total Players" : [uniqueplayer_count]})
#print data frame
uniqueplayer_df

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [20]:
#get number of unique items 
uniqueitems = purchase_data["Item ID"].nunique()

#get average price of purchased items
avgprice = purchase_data["Price"].mean()

#get total number of purchases
totalpurchases = purchase_data["Purchase ID"].count()

#get total revenue
sumrevenue = purchase_data["Price"].sum()

#put numbers into dataframe to print
purchase_summary_df = pd.DataFrame({"Number of Unique Items" : [uniqueitems],
                                   "Average Purchase Price" : ["${:,.2f}".format(avgprice)],
                                   "Total Number of Purchases" : [totalpurchases],
                                   "Total Revenue" : ["${:,.2f}".format(sumrevenue)]})
#print dataframe
purchase_summary_df

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [21]:
#get dataframe that only has each player once
unique_gen_df = purchase_data.drop_duplicates(subset = "SN", keep = "first", inplace = False)

#count how many total people purchased something
wholecount = unique_gen_df["Gender"].count()

#count how many purchasers there were by gender
unique_df = unique_gen_df.groupby(["Gender"]).count()

#calculate percent of whole for each gender and add to dataframe
unique_df["Percentage of Players"] = (unique_df["Purchase ID"] / wholecount) * 100

#format column to %
def format(x):
        return "{:.2f}%".format(x)
unique_df["Percentage of Players"] = unique_df["Percentage of Players"].apply(format)

#drop unneccessary columns,rename columns to immitate answer, and sort
unique_df = unique_df.drop(columns={"SN","Age","Item ID","Item Name","Price"})
unique_df = unique_df.rename(columns={"Purchase ID":"Total Count of Players"})
unique_df = unique_df.sort_values("Total Count of Players", ascending=False)

#print dataframe
unique_df

,Total Count of Players,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [22]:
#get dataframe of the only columns necessary for question
gender_needed = pd.DataFrame(purchase_data, columns=["SN","Gender","Purchase ID","Price"])

# Purchase Count
purcount_gen = gender_needed.groupby(["Gender"]).count()
# Average Purchase Price
avgprice_gen = gender_needed.groupby(["Gender"]).mean()
# Total Purchase Value
tvp_gen = gender_needed.groupby(["Gender"]).sum()

#merge each answer's data frames together
merge_gen = pd.DataFrame.merge(purcount_gen, avgprice_gen, on="Gender",suffixes=("_count","_avg"))
merge_gen2 = pd.DataFrame.merge(merge_gen, tvp_gen, on="Gender",suffixes=("_merge","_tvp"))

#clean up merged df by dropping and renaming columns
merge_gen2 = merge_gen2.drop(columns={"Purchase ID_count","Price_count","Purchase ID_avg","Purchase ID"})
merge_gen2 = merge_gen2.rename(columns={"SN":"Purchase Count","Price_avg":"Average Purchase Price","Price":"Total Purchase Value"})

#merge with df used in previous question to get number of unique purchasers
merge_gen3 = pd.DataFrame.merge(merge_gen2, unique_df, on="Gender")

# Average Purchase Total per Person by Gender
merge_gen3["Avg Total Purchase per Person"] = (merge_gen3["Total Purchase Value"] / merge_gen3["Total Count of Players"])

#clean up merged df by dropping unecessary columns
merge_gen3 = merge_gen3.drop(columns={"Total Count of Players"})

#format columns to $
def format(x):
        return "${:.2f}".format(x)
merge_gen3["Average Purchase Price"] = merge_gen3["Average Purchase Price"].apply(format)
merge_gen3["Total Purchase Value"] = merge_gen3["Total Purchase Value"].apply(format)
merge_gen3["Avg Total Purchase per Person"] = merge_gen3["Avg Total Purchase per Person"].apply(format)

#drop unecessary column
merge_gen3 = merge_gen3.drop(columns={"Percentage of Players"})

#print df
merge_gen3

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [23]:
#get dataframe that only has each player once
unique_age_df = purchase_data.drop_duplicates(subset = "SN", keep = "first", inplace = False)

#get max age
great = unique_age_df["Age"].max()
#set bins to range from 0 to the max age
bins = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, great]
#establish names for each bin
group_names = ["<10", "10-14", "15-19", "20-24", "25-29","30-34","35-39","40+"]
#give data a category based on age
unique_age_df["Age Bracket"] = pd.cut(unique_age_df["Age"], bins, labels=group_names, include_lowest=True)

#count total purchasers
wholecount = unique_age_df["Age"].count()

#get count for each age range
unique_age_df = unique_age_df.groupby(["Age Bracket"]).count()

#calculate percent of whole for each age range and add to dataframe
unique_age_df["Percentage of Players"] = (unique_age_df["Purchase ID"] / wholecount) * 100

#format column to %
def format(x):
        return "{:.2f}%".format(x)
unique_age_df["Percentage of Players"] = unique_age_df["Percentage of Players"].apply(format)

#drop unneccessary columns and rename columns to immitate answer
unique_age_df = unique_age_df.drop(columns={"SN","Age","Item ID","Item Name","Price","Gender"})
unique_age_df = unique_age_df.rename(columns={"Purchase ID":"Total Count of Players"})

#print df
unique_age_df

/Users/ashtonsmith/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,Total Count of Players,Percentage of Players
Age Bracket,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [24]:
#get dataframe of the only columns necessary for question
age_needed = pd.DataFrame(purchase_data, columns=["SN","Purchase ID","Price","Age"])

#get max age
great = age_needed["Age"].max()
#set bins to range from 0 to the max age
bins = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, great]
#establish names for each bin
group_names = ["<10", "10-14", "15-19", "20-24", "25-29","30-34","35-39","40+"]
#give data a category based on age
age_needed["Age Bracket"] = pd.cut(age_needed["Age"], bins, labels=group_names, include_lowest=True)

# Purchase Count
purcount_age = age_needed.groupby(["Age Bracket"]).count()
# Average Purchase Price
avgprice_age = age_needed.groupby(["Age Bracket"]).mean()
# Total Purchase Value
tvp_age = age_needed.groupby(["Age Bracket"]).sum()

#merge each answer's data frames together
merge_age = pd.DataFrame.merge(purcount_age, avgprice_age, on="Age Bracket",suffixes=("_count","_avg"))
merge_age2 = pd.DataFrame.merge(merge_age, tvp_age, on="Age Bracket",suffixes=("_merge","_tvp"))

#clean up merged df by dropping and renaming columns
merge_age2 = merge_age2.drop(columns={"Purchase ID_count","Price_count","Purchase ID_avg","Purchase ID"})
merge_age2 = merge_age2.rename(columns={"SN":"Purchase Count","Price_avg":"Average Purchase Price","Price":"Total Purchase Value"})

#merge with df used in previous question to get number of unique purchasers
merge_age3 = pd.DataFrame.merge(merge_age2, unique_age_df, on="Age Bracket")

# Average Purchase Total per Person by age range
merge_age3["Avg Total Purchase per Person"] = (merge_age3["Total Purchase Value"] / merge_age3["Total Count of Players"])

#clean up merged df by dropping unecessary columns
merge_age3 = merge_age3.drop(columns={"Total Count of Players","Percentage of Players","Age","Age_avg","Age_count"})

#formatted to $
def format(x):
        return "${:.2f}".format(x)
merge_age3["Average Purchase Price"] = merge_age3["Average Purchase Price"].apply(format)
merge_age3["Total Purchase Value"] = merge_age3["Total Purchase Value"].apply(format)
merge_age3["Avg Total Purchase per Person"] = merge_age3["Avg Total Purchase per Person"].apply(format)

#print df
merge_age3

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Bracket,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [25]:
#get dataframe of the only columns necessary for question
player_needitem_df = pd.DataFrame(purchase_data, columns=["SN","Purchase ID","Price"])

#purchase count
player_purchase_count = player_needitem_df.groupby("SN").count()
#average purchase price
player_tvp = player_needitem_df.groupby("SN").sum()
#total purchase value
player_average = player_needitem_df.groupby("SN").mean()

#merge answers into one data frame
player_merge1 = pd.DataFrame.merge(player_purchase_count, player_tvp, on="SN", suffixes=("_count","_tvp"))
player_merge2 = pd.DataFrame.merge(player_merge1, player_average, on="SN", suffixes=("","_average"))

#clean up merged df by dropping unecessary columns, renaming columns, and sorting
player_merge2 = player_merge2.drop(columns={"Purchase ID_tvp","Price_count","Purchase ID"})
player_merge2 = player_merge2.rename(columns={"Purchase ID_count":"Purchase Count","Price":"Average Purchase Price","Price_tvp":"Total Purchase Value"})
player_merge2 = player_merge2.sort_values("Total Purchase Value", ascending=False)

#resest df to only include certain columns
player_merge2 = player_merge2[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]

#format columns to $
def format(x):
        return "${:.2f}".format(x)
player_merge2['Average Purchase Price'] = player_merge2['Average Purchase Price'].apply(format)
player_merge2['Total Purchase Value'] = player_merge2['Total Purchase Value'].apply(format)

#print top 5 of data frame
player_merge2.head(5)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [26]:
#get dataframe of the only columns necessary for question
popneeditem_df = pd.DataFrame(purchase_data, columns=["Item ID","Item Name","Price"])

#total purchase value
pop_tvp = popneeditem_df.groupby(["Item ID","Item Name"]).sum()
#purchase count
pop_item_purcount = popneeditem_df.groupby(["Item ID","Item Name"]).count()
#item price
pop_itemprice = popneeditem_df.groupby(["Item ID","Item Name"]).mean()

#merge answers into one data frame
pop_merge1 = pd.DataFrame.merge(pop_tvp, pop_item_purcount, on=["Item ID","Item Name"])
pop_merge2 = pd.DataFrame.merge(pop_merge1, pop_itemprice, on=["Item ID","Item Name"])

#clean up merged df by dropping unecessary columns, renaming columns, and sorting
pop_merge2 = pop_merge2.rename(columns={"Price_x":"Total Purchase Value","Price_y":"Purchase Count","Price":"Item Price"})
pop_merge2 = pop_merge2[["Purchase Count","Item Price", "Total Purchase Value"]]
pop_merge2 = pop_merge2.sort_values("Purchase Count", ascending=False)

#format columns to $
def format(x):
    return "${:.2f}".format(x)
pop_merge2['Total Purchase Value'] = pop_merge2['Total Purchase Value'].apply(format)
pop_merge2['Item Price'] = pop_merge2['Item Price'].apply(format)

#print top 5 in df
pop_merge2.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [27]:
#get dataframe of the only columns necessary for question
needitem_df = pd.DataFrame(purchase_data, columns=["Item ID","Item Name","Price"])

#total purchase value
tvp = needitem_df.groupby(["Item ID","Item Name"]).sum()
#purchase count
item_purcount = needitem_df.groupby(["Item ID","Item Name"]).count()
#item price
itemprice = needitem_df.groupby(["Item ID","Item Name"]).mean()

#merge answers into one data frame
merge1 = pd.DataFrame.merge(tvp, item_purcount, on=["Item ID","Item Name"])
merge2 = pd.DataFrame.merge(merge1, itemprice, on=["Item ID","Item Name"])

#clean up merged df by dropping unecessary columns, renaming columns, and sorting
merge2 = merge2.rename(columns={"Price_x":"Total Purchase Value","Price_y":"Purchase Count","Price":"Item Price"})
merge2 = merge2[["Purchase Count","Item Price", "Total Purchase Value"]]
merge2 = merge2.sort_values("Total Purchase Value", ascending=False)

#format columns to $
def format(x):
    return "${:.2f}".format(x)
merge2['Total Purchase Value'] = merge2['Total Purchase Value'].apply(format)
merge2['Item Price'] = merge2['Item Price'].apply(format)

#print top 5 in df
merge2.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
